# Python PostgreSQL Connection and Query Wrapper

This Python script provides functionality to connect to a PostgreSQL database and includes a query wrapper for executing SQL queries. It utilizes the `psycopg2` library for database connection and the `dotenv` library for loading environment variables from a `.env` file.

## Prerequisites

Make sure to have the following installed:

- Python
- PostgreSQL
- `psycopg2` library (`pip install psycopg2`)
- `dotenv` library (`pip install python-dotenv`)

## Usage

1. Create a `.env` file in the same directory as the script with the following variables:

   ```env
   USER=your_database_user
   PASSWORD=your_database_password
   HOST=your_database_host
   PORT=your_database_port
   DATABASE=your_database_name


In [7]:
import psycopg2
from psycopg2 import OperationalError
import os
from dotenv import load_dotenv
load_dotenv()

url = os.getenv('URL')

def connect_to_database():
    try:
        connection = psycopg2.connect(
            user=os.getenv('USER')
            password=os.getenv('PASSWORD'),
            host=os.getenv('HOST'),
            port=os.getenv('PORT'),
            database=os.getenv('DATABASE'),
        )
        return connection

    except OperationalError as e:
        print(f"Error: {e}")
        return None

def close_connection(connection):
    if connection:
        connection.close()
        print("Connection closed.")

def sql_query(func):
    def wrapper(*args, **kwargs):
        connection = connect_to_database()
        try:
            result = func(connection, *args, **kwargs)
            return result
        finally:
            close_connection(connection)

    return wrapper

# PostgreSQL Queries

## Retrieval Queries

- **Get PostgreSQL Version**: Retrieves the version of PostgreSQL.
- **Get Current User**: Retrieves the current database user.
- **Get Current Timestamp**: Retrieves the current timestamp.
- **Get Available Databases**: Retrieves the names of available databases.
- **Get All Manufacturers**: Retrieves manufacturer names and page links.
- **Get All Models**: Retrieves model names and page links.
- **Get All Parts**: Retrieves part IDs and page links.

## Insertion Queries

- **Add Manufacturer**: Adds a new manufacturer if not already present.
- **Add Model**: Adds a new model if not already present.
- **Add Part Catalogue and Parts**: Adds part catalogues and associated parts.

Note: Functions use the `@sql_query` decorator for handling database connections.


In [8]:
@sql_query
def get_postgres_version(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT version();")
        return cursor.fetchall()

@sql_query
def get_current_user(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT current_user;")
        return cursor.fetchall()

@sql_query
def get_current_timestamp(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT current_timestamp;")
        return cursor.fetchall()

@sql_query
def get_available_databases(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT datname FROM pg_database;")
        return cursor.fetchall()

@sql_query
def get_all_manufacturers(connection):
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT name, page_link FROM manufacturer;")
            manufacturers = cursor.fetchall()
            return manufacturers

    except Exception as e:
        print(f"Error: {e}")
        return None

@sql_query
def get_all_models(connection):
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT name, page_link FROM model;")
            models = cursor.fetchall()
            return models
    except Exception as e:
        print(f"Error: {e}")
        return None

@sql_query
def get_all_parts(connection):
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT id, page_link FROM part;")
            parts = cursor.fetchall()
            return parts
    except Exception as e:
        print(f"Error: {e}")
        return None

In [9]:
from psycopg2 import IntegrityError

@sql_query
def add_manufacturer(connection, name, page_link):
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT 1 FROM manufacturer WHERE name = %s;", (name,))
            exists = cursor.fetchone()

            if not exists:
                cursor.execute("INSERT INTO manufacturer (name, page_link) VALUES (%s, %s);", (name, page_link))
                connection.commit()
                print(f"Manufacturer '{name}' added successfully.")
            else:
                print(f"Manufacturer with name '{name}' already exists. Skipping insertion.")

    except IntegrityError as e:
        print(f"IntegrityError: {e}")

@sql_query
def add_model(connection, model_data):
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT 1 FROM model WHERE name = %s;", (model_data['name'],))
            exists = cursor.fetchone()

            if not exists:
                cursor.execute("INSERT INTO model (name, img_link, year_start, year_end, page_link, manufacturer_name) VALUES (%s, %s, %s, %s, %s, %s);",
                               (model_data['name'], model_data['img'], model_data['year_start'], model_data['year_end'], model_data['res_link'], 
                               model_data['manufacturer_name']))
                connection.commit()
                print(f"Model '{model_data['name']}' added successfully.")
            else:
                print(f"Model with name '{model_data['name']}' already exists. Skipping insertion.")

    except IntegrityError as e:
        print(f"IntegrityError: {e}")

@sql_query
def add_part_catalogue_and_parts(connection, data):
    try:
        with connection.cursor() as cursor:
            model_name = data['model_name']
            part_catalogue_name = data['part_catalogue_name']
            parts_list = data['parts_list']

            cursor.execute("""
                INSERT INTO part_catalogue (name)
                VALUES (%s)
                ON CONFLICT (name) DO NOTHING;
            """, (part_catalogue_name,))

            for part in parts_list:
                cursor.execute("""
                    INSERT INTO part (name, page_link, model_name, catalogue_name)
                    VALUES (%s, %s, %s, %s);
                """, (part['title'], part['href'], model_name, part_catalogue_name))

            connection.commit()
            print(f"Parts for model '{model_name}' and catalogue '{part_catalogue_name}' added successfully.")

    except IntegrityError as e:
        print(f"IntegrityError: {e}")

@sql_query
def add_product_and_manufacturer(connection, product_data):
    try:
        with connection.cursor() as cursor:
            cursor.execute("INSERT INTO product_manufacturer (name) VALUES (%s) ON CONFLICT DO NOTHING;", (product_data['manufacturer_name'],))
            connection.commit()
            print(f"Product manufacturer '{product_data['manufacturer_name']}' added successfully.")

            cursor.execute("INSERT INTO product (code, price, part_id, manufacturer_name, description, city) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;",
                           (product_data['code'], product_data['price'], product_data['part_id'],
                            product_data['manufacturer_name'], product_data['description'], product_data['city']))
            connection.commit()
            print(f"Product '{product_data['code']}' added successfully.")

    except IntegrityError as e:
        print(f"IntegrityError: {e}")

This function performs a web request to the specified URL and returns a BeautifulSoup object if the response status is 200.


In [11]:
import requests
from bs4 import BeautifulSoup

def fetch(url: str):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser') if response.status_code == 200 else None

# Manufacturer Scraping and Database Insertion

## Overview

This code fetches manufacturer information from a URL, processes HTML content, and inserts data into the database.

## Fetching and Processing

1. **Fetching Manufacturers**: Fetch HTML content and identify manufacturer list.

2. **Extracting Details**: Extract manufacturer names and associated URLs.

## Database Insertion

3. **Adding Manufacturers to DB**: Use `add_manufacturer` to insert manufacturer data into the database.

4. **Associating URLs**: Combine base URL with fetched URLs for completeness.

## Conclusion

Efficiently scrapes manufacturer details and inserts them into the database for further analysis.


In [ ]:
soup = fetch(url)
li_mark_list = soup.find_all('ul')[3].find_all('li')
[add_manufacturer(li.a.strong.string, url + li.a['href'])
    for li in li_mark_list]

# Model Scraping and Database Insertion

## Overview

This script iterates through manufacturers, fetches model details from associated URLs, processes HTML content, and inserts data into the database.

## Manufacturer Iteration

1. **Fetching Manufacturers**: Get manufacturer names and associated URLs using `get_all_manufacturers`.

2. **Processing Models**: Iterate through manufacturers and fetch model details from each URL.

## Model Processing and Database Insertion

3. **Extracting Details**: Extract model-specific information such as image source, name, production years, and links.

4. **Link Adjustment**: Adjust links for completeness and consistency.

5. **Database Insertion**: Use `add_model` to insert model data into the database.

## Conclusion

Efficiently scrapes model details for each manufacturer and inserts them into the database for further analysis.


In [ ]:
for manufacturer_name, url in get_all_manufacturers():
    soup = fetch(url)
    articles = soup.find_all('article', class_='catalog-body__model')

    for article in articles:
        gen_href = article.find(class_='catalog-body__model-name').a.get('href')

        img_src    = article.find(class_="c-card__image").img.get('src')
        model_name = article.find(class_="c-card__body").get('title')
        model_year_start = int(article.find(class_="c-card__production-years").find_all('span')[0].text)
        model_year_end   = int(article.find(class_="c-card__production-years").find_all('span')[1].text)
        href = article.find(class_="c-card__body").get('href')

        link = href if href is not None else gen_href
        res_link = url + link[2:].replace('.', '')

        print(res_link)

        result = {
            'manufacturer_name': manufacturer_name,
            'img': img_src,
            'name': model_name,
            'year_start': model_year_start,
            'year_end': model_year_end,
            'res_link': res_link
        }

        add_model(result)

# Model Parts Scraping and Database Insertion

## Overview

This code fetches model parts information from provided URLs, processes HTML content, and inserts data into the database.

## Fetching and Processing

1. **Fetching Models**: Iterate through models, fetching HTML content for each.

2. **Extracting Components**: Identify parts components within each model's HTML.

3. **Extracting Details**: Extract manufacturer, model parts, and relevant details.

## Database Insertion

4. **Adding Parts to DB**: Use `add_part_catalogue_and_parts` to insert part data into the database.

5. **Associating Parts**: Associate parts with the model in the `part` table.

6. **Limiting Part Types**: Include only the first three part types.

## Conclusion

Efficiently scrapes and associates model parts with the database for analysis and retrieval.


In [ ]:
for model_name, url in get_all_models():
    soup = fetch(url)
    model_components = soup.find_all(class_='catalog-model__body-content')

    manufacturer = soup.find('h1', class_='catalog-model__title').text.replace('Каталог запчастей ', '')

    model_parts_list = list()

    for model_component in model_components:
        c_card = model_component.find(class_='c-card')
        c_card_info = c_card.find(class_='c-card__info')
        c_card_img = c_card.find(class_='c-card__image').img.get('src')

        model_parts = model_component.find(class_='catalog-model__parts-block')

        panel_components = model_parts.find_all(class_='catalog-model__panel')
        part_components = model_parts.find_all('li', class_='catalog-model__parts-item')

        part_types_list = [
        {'title': part_component.a.get('title'), 
            'href': url + part_component.a.get('href')} 
        for part_component in part_components ]

        ext_txt = lambda x: x.h2.text
        panel_component_text_list = [
            add_part_catalogue_and_parts({
                'model_name': model_name,
                'part_catalogue_name': ext_txt(panel_component),
                'parts_list': part_types_list[:3]
            }) for panel_component in panel_components]

# Product Scraping and Database Insertion

## Overview

This script iterates through parts, fetches product details from associated URLs, processes HTML content, and inserts data into the database.

## Part Iteration

1. **Fetching Parts**: Get part IDs and associated URLs using `get_all_parts`.

2. **Processing Products**: Iterate through parts and fetch product details from each URL.

## Product Processing and Database Insertion

3. **Extracting Details**: Extract product-specific information such as code, manufacturer name, price, city, and description.

4. **Database Insertion**: Use `add_product_and_manufacturer` to insert product data into the database.

## Conclusion

Effectively scrapes product details for each part and inserts them into the database for further analysis.


In [ ]:
for part_id, url in get_all_parts():
    soup = fetch(url)
    colgroup = soup.find('colgroup')
    attributes = { 'data-seller-is-safe-deal': 'True', 'data-wholesale': '0' }
    trs = soup.find_all('tr', attrs=attributes)

    [ add_product_and_manufacturer({
        'code': tr.find('td', attrs={'data-type': 'code'}).text.strip(),
        'manufacturer_name': tr.find('td', attrs={'data-type': 'maker'}).text.strip(),
        'price': float(tr.find('td', attrs={'data-type': 'price'}).get('data-value')),
        'city': tr.find('td', attrs={'data-type': 'delivery'}).get('data-city'),
        'description': tr.find_all('td')[3].get('title'),
        'part_id': part_id
    }) for tr in trs]